# Getting Started: pyutils Basics with Dask

## Introduction

This tutorial builds on the basic `pyutils` functionality from [pyutils_basics.ipynb](pyutils_basics.ipynb) but focuses on **parallel processing** of multiple files using **DaskProcessor**.

You will learn how to:
- Use `DaskProcessor` to process multiple files in parallel
- Apply the same analysis to aggregated data from many files
- Scale from a single file to thousands of files without code changes
- Leverage multi-core systems and HPC clusters

## Key Difference from Serial Processing

| Aspect | Standard `Processor` | `DaskProcessor` |
|--------|----------------------|-----------------|
| **Best For** | Single file or small file lists | Multiple files / large datasets |
| **Parallelization** | Thread pool on single machine | Dask distributed framework |
| **Scaling** | Limited to machine cores | Scales to clusters |
| **Progress Tracking** | Basic logging | Real-time progress bar |

## Table of Contents
1. Setting up your environment
2. Creating a file list for multi-file processing
3. Processing data with DaskProcessor
4. Applying selection cuts to aggregated data
5. Inspecting aggregated data
6. Performing vector operations
7. Creating plots from aggregated results
8. Complete example with multiple files

## 1. Setting Up Your Environment

In [ ]:
# Import external packages
import awkward as ak
import numpy as np

# Import pyutils modules
from pyutils.pydask import DaskProcessor
from pyutils.pyselect import Select
from pyutils.pyprint import Print
from pyutils.pyvector import Vector
from pyutils.pyplot import Plot
from pyutils.pylogger import Logger

# Initialize logger for notebook output
logger = Logger(print_prefix="[pyutils_dask]", verbosity=2)

## 2. Creating a File List for Multi-File Processing

When working with multiple files, you need to create a file list. Each line in the file should contain the full path to a ROOT file.

In [ ]:
import os

# Load the MDS3a.txt file list from the repository
notebook_dir = os.getcwd()
file_list_path = os.path.join(notebook_dir, "../../MDS3a.txt")

# Read the file list
if os.path.exists(file_list_path):
    with open(file_list_path, 'r') as f:
        sample_files = [line.strip() for line in f if line.strip()]
    
    logger.log(f"Loaded file list from: MDS3a.txt", "success")
    logger.log(f"Total files available: {len(sample_files)}", "info")
else:
    logger.log("MDS3a.txt not found, creating empty list", "warning")
    sample_files = []

# Display first few files
logger.log("First 3 files in the list:", "info")
for i, f in enumerate(sample_files[:3]):
    logger.log(f"  {i+1}. {f.split('/')[-1]}", "info")

## 3. Processing Data with DaskProcessor

Now process all files in parallel using DaskProcessor. This demonstrates the key advantage of Dask: as your file list grows from 1 to 1000s of files, the same code automatically scales across available cores or connects to a remote HPC cluster.

In [ ]:
# Initialize DaskProcessor with remote settings for multi-file processing
processor = DaskProcessor(
    tree_path="EventNtuple/ntuple",
    use_remote=True,           # Access remote persistent datasets
    location="disk",           # Read from disk storage
    verbosity=1,
    worker_verbosity=0
)

logger.log("DaskProcessor initialized", "success")
logger.log("Ready to process multiple files in parallel", "info")

# Define branches to extract
branches = ["trksegs"]

logger.log(f"Branches to extract: {branches}", "info")
logger.log("Reading from persistent dataset (disk location)", "info")

In [ ]:
# Process data using DaskProcessor with multiple files in parallel
logger.log("Processing multiple files with DaskProcessor...", "info")
logger.log(f"Total files to process: {len(sample_files)}", "info")

try:
    if sample_files and file_list_path:
        logger.log("Using DaskProcessor with multi-file parallel processing", "info")
        
        # Process all files in parallel
        data = processor.process_data(
            file_list_path=file_list_path,
            branches=branches,
            n_workers=4,              # Use 4 parallel workers
            threads_per_worker=1,     # 1 thread per worker
            processes=False,          # Use threads (not processes)
            show_progress=True        # Show progress bar during processing
        )
        
    else:
        logger.log("File list not available, processing single file for demo", "warning")
        
        from pyutils.pyprocess import Processor
        demo_processor = Processor(
            verbosity=1,
            use_remote=True,
            location="disk"
        )
        
        # Use first file from MDS3a.txt list
        if sample_files:
            demo_file = sample_files[0]
            logger.log(f"Processing first file from MDS3a.txt: {demo_file.split('/')[-1]}", "info")
        else:
            demo_file = "/pnfs/mu2e/tape/phy-nts/nts/mu2e/MDS2ac-OnSpillTriggered/MDC2020aw_perfect_v1_3/root/8c/0b/nts.mu2e.MDS2ac-OnSpillTriggered.MDC2020aw_perfect_v1_3.0.root"
            logger.log("Using fallback file for demo", "warning")
        
        data = demo_processor.process_data(
            file_name=demo_file,
            branches=branches
        )
    
    logger.log("Data aggregation complete", "success")
    logger.log(f"Total events from all files: {len(data)}", "info")
    
except Exception as e:
    logger.log(f"Error during processing: {e}", "error")
    logger.log("Check that Mu2e environment is properly initialized", "warning")

## 4. Applying Selection Cuts to Aggregated Data

After processing all files with DaskProcessor, apply selection cuts to the combined dataset. The workflow is identical to the serial case.

In [ ]:
# Initialize selector
selector = Select(verbosity=1)

# Select track segments at tracker entrance
logger.log("Selecting tracks at tracker entrance...", "info")

at_trkent = selector.select_surface(
    data=data,
    surface_name="TT_Front"
)

# Add selection mask to data
data["at_trkent"] = at_trkent

# Apply mask
trkent = data[at_trkent]

logger.log(f"Selected {len(trkent)} events at tracker entrance", "success")
logger.log(f"Selection efficiency: {100*len(trkent)/len(data):.1f}%", "info")

## 5. Inspecting Aggregated Data

Verify the data structure and cuts using the Print utility.

In [ ]:
# Initialize printer
printer = Print(verbose=False)

# Display data structure after selection
logger.log("Data structure at tracker entrance:", "info")
printer.print_n_events(trkent, n_events=1)

## 6. Performing Vector Operations on Aggregated Data

Calculate quantities (like momentum magnitude) from the aggregated data.

In [ ]:
# Initialize vector processor
vector = Vector(verbosity=1)

# Calculate momentum magnitude for all tracks
logger.log("Computing momentum magnitude...", "info")

mom_mag = vector.get_mag(
    branch=trkent["trksegs"],
    vector_name="mom"
)

logger.log("Momentum magnitude computed", "success")
logger.log(f"Momentum range: {ak.min(ak.flatten(mom_mag, axis=None)):.2f} - {ak.max(ak.flatten(mom_mag, axis=None)):.2f} MeV/c", "info")

## 7. Creating Plots from Aggregated Results

Create publication-quality plots from the combined dataset. These plots show aggregated statistics across all processed files.

In [ ]:
# Initialize plotter
plotter = Plot()

# Flatten arrays for plotting
time_flat = ak.flatten(trkent["trksegs"]["time"], axis=None)
mom_mag_flat = ak.flatten(mom_mag, axis=None)

logger.log(f"Preparing plots from {len(time_flat)} track measurements", "info")

### 7.1 Create 1D Histogram: Time Distribution

In [ ]:
logger.log("Creating 1D histogram of time distribution...", "info")

plotter.plot_1D(
    time_flat,
    nbins=100,
    xmin=450,
    xmax=1695,
    title="Time at Tracker Entrance (from Dask-processed Data)",
    xlabel="Fit time at Trk Ent [ns]",
    ylabel="Events per bin",
    out_path='h1_time_dask.png',
    stat_box=True,
    error_bars=True
)

logger.log("1D histogram created: h1_time_dask.png", "success")

: 

### 7.2 Create 2D Histogram: Momentum vs. Time

In [ ]:
logger.log("Creating 2D histogram of momentum vs. time...", "info")

plotter.plot_2D(
    x=mom_mag_flat,
    y=time_flat,
    nbins_x=100,
    xmin=85,
    xmax=115,
    nbins_y=100,
    ymin=450,
    ymax=1650,
    title="Momentum vs. Time at Tracker Entrance (from Dask-processed Data)",
    xlabel="Fit mom at Trk Ent [MeV/c]",
    ylabel="Fit time at Trk Ent [ns]",
    out_path='h2_timevmom_dask.png'
)

logger.log("2D histogram created: h2_timevmom_dask.png", "success")

## 8. Complete Example with Multiple Files

Here's a complete working example showing how to scale from a single file to many files using DaskProcessor. The same analysis code works for any number of files.

In [ ]:
"""
COMPLETE EXAMPLE: Multi-File Analysis with DaskProcessor

The following code is a template for analyzing multiple files.
Replace file_list_path with your actual file list to run the analysis.

Key features:
- Processes multiple files in parallel using Dask
- Same analysis code as above - just swap the Processor call
- Automatically scales to available cores/clusters
- Built-in progress monitoring
"""

def complete_dask_analysis(file_list_path, branches, n_workers=4):
    """
    Complete workflow for multi-file analysis with DaskProcessor.
    
    Parameters:
        file_list_path (str): Path to file containing list of ROOT files
        branches (list): List of branches to extract
        n_workers (int): Number of Dask workers to use
    
    Returns:
        dict: Results including plots and statistics
    """
    
    # 1. INITIALIZE
    processor = DaskProcessor(tree_path="EventNtuple/ntuple", verbosity=1)
    selector = Select(verbosity=0)
    vector = Vector(verbosity=0)
    plotter = Plot()
    
    # 2. PROCESS DATA - This is where DaskProcessor scales across files!
    data = processor.process_data(
        file_list_path=file_list_path,
        branches=branches,
        n_workers=n_workers,
        threads_per_worker=1,
        processes=False,
        show_progress=True  # Progress bar during processing
    )
    
    if data is None:
        print("Error: No data returned from processing")
        return None
    
    # 3. APPLY SELECTIONS
    at_trkent = selector.select_surface(data=data, surface_name="TT_Front")
    trkent = data[at_trkent]
    
    # 4. COMPUTE QUANTITIES
    mom_mag = vector.get_mag(branch=trkent["trksegs"], vector_name="mom")
    
    # 5. CREATE PLOTS
    time_flat = ak.flatten(trkent["trksegs"]["time"], axis=None)
    mom_mag_flat = ak.flatten(mom_mag, axis=None)
    
    # 1D plot
    plotter.plot_1D(
        time_flat, nbins=100, xmin=450, xmax=1695,
        title="Time Distribution", xlabel="Time [ns]",
        ylabel="Events", out_path='time_dask.png'
    )
    
    # 2D plot
    plotter.plot_2D(
        x=mom_mag_flat, y=time_flat,
        nbins_x=100, xmin=85, xmax=115,
        nbins_y=100, ymin=450, ymax=1650,
        title="Momentum vs. Time", 
        xlabel="Momentum [MeV/c]", ylabel="Time [ns]",
        out_path='momentum_time_dask.png'
    )
    
    # 6. RETURN RESULTS
    return {
        'total_events': len(data),
        'selected_events': len(trkent),
        'efficiency': len(trkent) / len(data),
        'mom_min': ak.min(mom_mag_flat),
        'mom_max': ak.max(mom_mag_flat),
        'time_min': ak.min(time_flat),
        'time_max': ak.max(time_flat),
    }


# Example usage (uncomment and modify for real analysis):
# 
# # Create your file list (one file per line)
# file_list = "/path/to/file_list.txt"
# 
# # Run analysis
# results = complete_dask_analysis(
#     file_list_path=file_list,
#     branches=["trksegs"],
#     n_workers=4
# )
# 
# # Print results
# if results:
#     print(f"Total events: {results['total_events']}")
#     print(f"Selected events: {results['selected_events']}")
#     print(f"Selection efficiency: {100 * results['efficiency']:.1f}%")

logger.log("Complete example function defined", "success")
logger.log("See commented code above for usage instructions", "info")

## Key Takeaways

### When to Use DaskProcessor

Use **DaskProcessor** (this tutorial) when:
- ✓ Processing multiple files (10s, 100s, or 1000s)
- ✓ You want progress monitoring across files
- ✓ You want automatic scaling to available cores
- ✓ You plan to connect to an HPC cluster later
- ✓ You need resilience to transient failures

Use **Standard Processor** when:
- ✓ Working with a single file
- ✓ Processing a small number of files (< 5)
- ✓ Simpler debugging is needed
- ✓ Faster startup time is important

### Workflow Comparison

```
STANDARD PROCESSOR           DASK PROCESSOR
──────────────────────────   ──────────────────────────
File 1 → Process             File 1 ─┐
File 2 → Process        or        File 2 ─┼→ Parallel Processing
File 3 → Process             File 3 ─┘
                             (scales across cores/cluster)

Same analysis code after processing!
```

### Scaling Path

1. **Single File**: Use standard `Processor` for development
2. **Multiple Files**: Switch to `DaskProcessor` with local workers
3. **Large Scale**: Point to remote Dask scheduler on HPC cluster
4. **Code Change**: None! Just change the scheduler address

The real power of DaskProcessor is that you write your analysis once, and it scales effortlessly from your laptop to a full cluster.